<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/homework/Dz_Pro35_%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D1%80%D0%B5%D1%87%D0%B8_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82_%D0%BE%D1%82%D0%B2%D0%B5%D1%87%D0%B0%D1%8E%D1%89%D0%B8%D0%B9_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%BC_%D0%B8_%D0%B3%D0%BE%D0%BB%D0%BE%D1%81%D0%BE%D0%BC__%D1%81_%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%BC_%D1%81%D0%B5%D1%82%D0%B8_%D1%8F%D0%BF%D0%BE%D0%BD%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном задании уровня Pro вам необходимо построить простого нейро-сотрудника на базе знаний компании Simble таким образом чтобы получить ответ от модели GPT в виде текста и в виде озвученного текста с параметром сети японского языка.

Ссылка на базу знаний - https://docs.google.com/document/d/1ova0gprJGQ7_wYJhxEozpzZTgsGYYgyADeNOm-l7rfM

In [ ]:
!pip install faiss-cpu langchain==0.0.271 openai tiktoken

In [ ]:
!pip install openai==0.28.0

In [ ]:
!pip install gTTS

In [ ]:
import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document
from googletrans import Translator
from gtts import gTTS
import IPython
from IPython.display import Audio
from IPython.display import Audio, display
from io import BytesIO
import ipywidgets as widgets

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1ova0gprJGQ7_wYJhxEozpzZTgsGYYgyADeNOm-l7rfM') # заполните параметр функции

In [ ]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/19a9uVTNbpQt8ggoFBC8cBYHiHFdMboWAGansFUq559M/edit?usp=drive_link') # заполните параметр ссылкой на составленный вами промпт. Не забудьте открыть доступ

In [ ]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))
print("Общее количество чанков: ", len(source_chunks))

Общее количество чанков:  97


In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [ ]:
# функции для приема вопросов и ответов
def answer_index(system, topic, search_index, temp=1, verbose=1):
    translator = Translator()

    docs = search_index.similarity_search(topic, k=5)
    message_content = '\n'.join([f'Отрывок документа №{i+1}\n=====================\n{doc.page_content}\n' for i, doc in enumerate(docs)])

    user_query = f"Ответь на вопрос. Документ с информацией для ответа:\n{message_content}\n\nВопрос клиента:\n{topic}"

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_query}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )

    response = insert_newlines(completion.choices[0].message.content)
    print("Ответ на русском языке:")
    print(response)  # Показываем ответ на русском языке

    # Переводим ответ на японский язык
    translation = translator.translate(response, src='ru', dest='ja')
    japanese_text = translation.text

    # Озвучиваем ответ на японском языке
    tts = gTTS(text=japanese_text, lang='ja')
    speech_bytes = BytesIO()
    tts.write_to_fp(speech_bytes)
    speech_bytes.seek(0)

    os.system(f'ffmpeg -i pipe:0 -f mp3 -acodec pcm_s16le -ar 44100 -ac 2 pipe:1 | aplay')  # Воспроизводим озвучку ответа на японском языке

    return response  # Возвращает ответ на русском языке





In [ ]:
# запускаем нейросотрудника. Если вы задали все интересующие вопросы - наберите в поле для вопросов -  стоп.  и диалог остановится.
def generate_speech(text, lang='ja'):
    tts = gTTS(text=text, lang=lang)
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

def get_response(question):
    if question.lower() == 'стоп':
        print("Диалог завершен. Спасибо за общение!")
        return False

    answer = answer_index(system, question, db)
    print("Ответ на ваш вопрос:", answer)
    generate_speech(answer, lang='ja')

    return True

# Функция для обработки введенного пользователем текста и получения ответа
def handle_submit(sender):
    user_question = text_input.value
    continue_dialogue = get_response(user_question)

    text_input.value = ""
    if not continue_dialogue:
        text_input.close()
        return

# Создание текстового виджета для ввода вопросов
text_input = widgets.Text(placeholder='Введите ваш вопрос на русском', description='Вопрос:')
text_input.on_submit(handle_submit)
display(text_input)


Text(value='', description='Вопрос:', placeholder='Введите ваш вопрос на русском')

Ответ на русском языке:
 Да, вы можете застраховать свою машину в компании Simble. Simble предоставляет страховые услуги на короткий период, включая страхование на поездку, километры в пути или  парковку. Чтобы оформить страховку в компании Simble, вам необходимо скачать приложение на устройство с операционной системой iOS или Android, указать свои данные и  данные автомобиля, а затем сфотографировать автомобиль со всех сторон. После этого выберите подходящий тариф и оформите полис. Simble также предлагает страховку ОСАГО по  выгодным тарифам.
Ответ на ваш вопрос:  Да, вы можете застраховать свою машину в компании Simble. Simble предоставляет страховые услуги на короткий период, включая страхование на поездку, километры в пути или  парковку. Чтобы оформить страховку в компании Simble, вам необходимо скачать приложение на устройство с операционной системой iOS или Android, указать свои данные и  данные автомобиля, а затем сфотографировать автомобиль со всех сторон. После этого выберите 

Ответ на русском языке:
 Да, вы можете застраховать машину на 5 дней в компании Simble. Simble предлагает страховку на короткий период, которая позволяет страховать автомобиль не только на 1 час  поездки или на время парковки, но и на 5 дней. Вам нужно оформить страховку на соответствующий период времени, выбрав подходящий тариф, который удовлетворит вашу  потребность.
Ответ на ваш вопрос:  Да, вы можете застраховать машину на 5 дней в компании Simble. Simble предлагает страховку на короткий период, которая позволяет страховать автомобиль не только на 1 час  поездки или на время парковки, но и на 5 дней. Вам нужно оформить страховку на соответствующий период времени, выбрав подходящий тариф, который удовлетворит вашу  потребность.


Ответ на русском языке:
 Извините, я не могу определить стоимость страховки на ваш автомобиль без дополнительных данных. Для расчета стоимости страховки необходимо учитывать не только марку и  год выпуска автомобиля, но и другие факторы, такие как регион использования автомобиля, стаж водителя, виды покрытий и тарифы страховых компаний. Рекомендую обратиться в  страховую компанию, предоставив все необходимые данные о вашем автомобиле и водительских характеристиках, чтобы получить точный расчет стоимости страховки на 10 дней для  вашего автомобиля VAZ 21031 выпущенного в 2017 году.
Ответ на ваш вопрос:  Извините, я не могу определить стоимость страховки на ваш автомобиль без дополнительных данных. Для расчета стоимости страховки необходимо учитывать не только марку и  год выпуска автомобиля, но и другие факторы, такие как регион использования автомобиля, стаж водителя, виды покрытий и тарифы страховых компаний. Рекомендую обратиться в  страховую компанию, предоставив все необходимые дан

Диалог завершен. Спасибо за общение!
